# Aprendizaje Supervisado
# Clasificación Parte 1
## Nicolás Salusso - 10710

#### 1) Se declara la variable **my_random** que contiene mi número de alumno. 

In [1]:
my_random = 14

#### 2) Implementacion de un clasificador kNN

2.1 Se carga el dataset

In [3]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()

X = breast_cancer.data

2.3 El target del dataset puede tomar 2 valores:
- Benigno
- Maligno

In [4]:
y = breast_cancer.target

2.4 Se comienza a instanciar el clasificador

In [5]:
from sklearn import neighbors

# Para el n_neighbors se tienen que ir probando

clf = neighbors.KNeighborsClassifier(n_neighbors=8, weights='distance')

#### 3) Se aplica z-score

In [6]:
from scipy import stats

X_norm = stats.zscore(X)

3.1 Se divide en conjunto de entrenamiento y de test

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, random_state=my_random, test_size=0.3)

3.2 Se entrena el modelo y se obtiene la tasa de aciertos del clasificador

In [8]:
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
print('Porcentaje de aciertos en la clasificación: ', clf.score(X_test,y_test))

Porcentaje de aciertos en la clasificación:  0.970760233918


In [9]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_hat, labels=[0, 1]))

[[ 59   4]
 [  1 107]]


In [10]:
print(metrics.classification_report(y_test, y_hat, target_names=breast_cancer.target_names))

             precision    recall  f1-score   support

  malignant       0.98      0.94      0.96        63
     benign       0.96      0.99      0.98       108

avg / total       0.97      0.97      0.97       171



4) Se le va a dar un costo a las salidas del clasificador para que este disminuya el error lo mas posible.


In [11]:
prob = clf.predict_proba(X_test)
# prob = prob.reshape(1, -1)
print(prob[1, 1])

1.0


In [14]:
for i in prob:
    if i[0] == 0 or i[0] == 1:
        pass
    elif i[1] < 0.8 and i[0] < 0.5:
        i[0] = None
        i[1] = None

print(prob)

# print(metrics.confusion_matrix(y_test, prob, labels=[0, 1]))

[[ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.13089299  0.86910701]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.62877938  0.37122062]
 [        nan         nan]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.86945794  0.13054206]
 [ 0.          1.        ]
 [ 1.          0.        ]
 [ 0.85026475  0.14973525]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.61669368  0.38330632]
 [ 0.          1.        ]
 [        nan         nan]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]
 